In [1]:
import googleapiclient.discovery
from oauth2client.client import GoogleCredentials

In [18]:
import argparse
import googleapiclient
from googleapiclient import discovery
from six.moves import input
from pprint import pprint
import time
import subprocess
import json
import argparse
import googleapiclient
from googleapiclient import discovery
from six.moves import input
import time
import os

import shlex
from google.cloud import storage
import os
from google.cloud import *
#path_to_key = '/Users/pgreisen/Projects/Enevolv/keys_gcp/ngsdataanalysis-08329a276003.json'
path_to_key = '/Users/pgreisen/Projects/Enevolv/keys_gcp/ngsdataanalysis-b3db1f51b89f.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_key

In [19]:
try:
    from oauth2client.client import GoogleCredentials  # pylint: disable=g-import-not-at-top
    import google.auth.exceptions  # pylint: disable=g-import-not-at-top
    from googleapiclient import discovery  # pylint: disable=g-import-not-at-top
    from googleapiclient import errors  # pylint: disable=g-import-not-at-top
except ImportError:
    print("Warning, google-api-python-client or google-auth not installed, can't connect to GCP instances.")

In [20]:
try:
    credentials = GoogleCredentials.get_application_default()
except google.auth.exceptions.DefaultCredentialsError:
    print("No credentials set")

In [21]:
def get_billing_id(cloud_billing_client):
    # Get billing account
    billing_result = cloud_billing_client.billingAccounts().list().execute()
    print("Billing result: ",billing_result)
    return billing_result["billingAccounts"][0]['name']

In [22]:
def get_project_list(cloud_billing_client,billing_id):
    # Get list of all projects in the billing id
    project_list = []
    project_result = cloud_billing_client.billingAccounts().projects().list(name=billing_id).execute()
    for project_row in project_result["projectBillingInfo"]:
        project_list.append(project_row["projectId"])
    return project_list

In [23]:
def get_instance_data(compute_client, project_list):
    # Get list & details of all instances for all projects in the project list
    output = []
    for project_id in project_list:
        # First get the list of zones
        zone_list = []
        zone_result = compute_client.zones().list(project=project_id).execute()
        for zone_rows in zone_result['items']:
            zone_list.append(zone_rows['name'])
        for zone_name in zone_list:
            # For every zone for each project, get the instance details
            instances_result = compute_client.instances().list(project=project_id, zone=zone_name).execute()
            if 'items' in instances_result: # Get instance details only when there are instances in the given zone
                for instance_row in instances_result["items"]:
                    output_row = []
                    metadata_dict = {}
                    output_row.append(project_id) # Project ID as the first field
                    output_row.extend([instance_row["name"],instance_row["labels"]["app-name"],instance_row["labels"]["biz-unit"],instance_row["labels"]["env-name"]])
                    # Get nested metadat key-value entries
                    for metadata_entry in instance_row["metadata"]["items"]:
                        metadata_dict[metadata_entry['key']] = metadata_entry['value']
                    output_row.extend([metadata_dict['server-role'], metadata_dict['server-type'], metadata_dict['os-image']])
                    output.append(output_row)
    return output

In [24]:
def write_output(header_row,output):
    # Write CSV format output to stdout
    print(header_row)
    for row in output:
        print(','.join(row))

In [25]:
def main():
    # Change the header row based on your requirement
    header_row = "project_id,name,labels-app-name,labels-biz-unit,labels-env-name,metadata-server-role,metadata-server-type,metadata-os-image"
    # Use default credentials
    credentials = GoogleCredentials.get_application_default()
    # Build and initialize the API
    compute_client = googleapiclient.discovery.build('compute', 'v1', credentials=credentials)
    cloud_billing_client = googleapiclient.discovery.build('cloudbilling', 'v1', credentials=credentials)

    billing_id = get_billing_id(cloud_billing_client) # get billing id
    project_list = get_project_list(cloud_billing_client,billing_id) # get project list for the given billing id
    output = get_instance_data(compute_client,project_list) # get instance details for the list of projects
    write_output(header_row,output)


if __name__ == "__main__":
    main()

Billing result:  {}


KeyError: 'billingAccounts'